##### Introduction to DSPy
Dspy allows us to structurally encode the behaviour of our foundation models, it defines a structural way to program instructions into foundation models, and optimizes our instructions for a particular model based on a defined metric

In [2]:
import dspy
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric

In [3]:
turbo = dspy.OpenAI(model="gpt-3.5-turbo-instruct", max_tokens=250)
dspy.settings.configure(lm=turbo)

In [5]:
# load dataset
gsm8k = GSM8K()
gsm8k_trainset, gsm8k_devset = gsm8k.train[:10], gsm8k.test[:10]

HfHubHTTPError: 501 Server Error: Not Implemented for url: https://huggingface.co/api/datasets/gsm8k

##### Define the module
With our environments setup, we will now define a custom program that uses ChainOfThought module to perform step by step reasoning to generate answers


In [6]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought("question -> answer")
    
    def forward(self, question):
        return self.prog(question)

##### Compile and Evaluate model
Now that we have defined our module and our dataset, we can then compile the moodel, it is during the compilation step where the optimizations take place. We also define a metric for optimizing the model. We will be performing optimiztions using the `BootstrapFewShotWithRandomSearch` teleprompter. 

In [ ]:
from dspy.teleprompt import BootstrapFewShot
config = dict(max_bootstrapped_demos=4, max_labelled_demos=4)

teleprompter = BootstrapFewShot(metric=gsm8k_metric, **config)
optimized_cot = teleprompter.compile(CoT(), trainset=gsm8k_trainset, valset=gsm8k_devset)


##### Evaluate 
Now that we have a compiled(optimized) DSPy program, let's move to evaluating its performance of the dev dataset


In [ ]:
from dspy.evaluate import Evaluate

evaluate = Evaluate(devset=gsm8k_devset, metric=gsm8k_metric, num_threads=4, 
                    display_progress=True, display_table=0)

evaluate(optimized_cot)